In [5]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import os

# analyze_drain_robust_background_removal 함수는 기존과 동일하게 유지
# (이전에 제공된 analyze_drain_robust_background_removal 함수 내용이 여기에 들어가야 합니다.)
def analyze_drain_robust_background_removal(image_path, use_roi=True):
    # 1) HSV/BGR thresholds (완전 까만 빈 부분 제외에 초점)
    brown_lower = np.array([0,  20,  50])   # V 10에서 50으로 올림
    brown_upper = np.array([70, 255, 255])
    
    gray_lower  = np.array([0,   40,  80])   # S 10에서 40으로, V 30에서 80으로 대폭 상향
    gray_upper  = np.array([180,100,200])    # V 상한 유지

    pack_lower  = np.array([0,   80,  50])
    pack_upper  = np.array([180,255,255])
    
    green_lower = np.array([30, 20, 10])
    green_upper = np.array([90, 255, 255])

    white_lower_bgr = np.array([100,100,100]) # 60에서 100으로 다시 올림
    white_upper_bgr = np.array([255,255,255])

    hsv_white_lower = np.array([0, 0, 100])     # V 80에서 100으로 올림
    hsv_white_upper = np.array([180, 50, 255])


    # 2) Load image
    full_image = cv2.imread(image_path)
    if full_image is None:
        print(f"Cannot load image: {image_path}")
        return None, None 

    h_full, w_full = full_image.shape[:2]
    
    # --- ROI 적용 여부 결정 (70% ROI) ---
    if use_roi:
        roi_width = int(w_full * 0.7)  # 너비 70%
        roi_height = int(h_full * 0.7) # 높이 70%
        x_start = int((w_full - roi_width) / 2)
        y_start = int((h_full - roi_height) / 2)
        roi = full_image[y_start : y_start + roi_height, x_start : x_start + roi_width]
        if roi.size == 0:
            print(f"ROI is empty for image: {image_path}. Check ROI dimensions.")
            return None, None
    else:
        roi = full_image.copy()
        x_start, y_start = 0, 0 
        roi_width, roi_height = w_full, h_full

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # 3) Build masks
    mask_brown = cv2.inRange(hsv, brown_lower, brown_upper)
    mask_gray  = cv2.inRange(hsv, gray_lower,  gray_upper)
    soil_mask  = cv2.bitwise_or(mask_brown, mask_gray)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)

    pack_mask  = cv2.inRange(hsv, pack_lower, pack_upper)
    pack_mask  = cv2.bitwise_and(pack_mask, cv2.bitwise_not(soil_mask)) 
    pack_mask = cv2.morphologyEx(pack_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    white_mask_bgr = cv2.inRange(roi, white_lower_bgr, white_upper_bgr) 
    white_mask_hsv = cv2.inRange(hsv, hsv_white_lower, hsv_white_upper)
    
    white_candidates_mask = cv2.bitwise_or(white_mask_bgr, white_mask_hsv)

    white_candidates_clean = cv2.morphologyEx(white_candidates_mask,
                                              cv2.MORPH_OPEN,
                                              np.ones((2,2), np.uint8)) 
    
    contours_white, _ = cv2.findContours(white_candidates_clean,
                                         cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)
    
    cig_mask = np.zeros_like(white_candidates_clean) 

    for c in contours_white:
        x,y,w,h = cv2.boundingRect(c)
        area = cv2.contourArea(c)

        if (h > w * 2.5 and area > 80) or (w > h * 2.5 and area > 80): # 담배 형태 필터링
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)
        elif area > 400 and h/w < 2 and w/h < 2: # 휴지 등
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)


    lbp_image = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    lbp_threshold = 60 # LBP 임계값 (빈 공간/그림자 질감 확실히 제외)
    lbp_mask = (lbp_image > lbp_threshold).astype(np.uint8) * 255
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)


    combined_initial_mask = cv2.bitwise_or(soil_mask, pack_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, green_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, cig_mask) 
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, lbp_mask) 

    kernel_small = np.ones((3,3), np.uint8)
    kernel_medium = np.ones((5,5), np.uint8)
    
    processed_mask = cv2.morphologyEx(combined_initial_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_CLOSE, kernel_medium, iterations=1) # 최종 Closing 강도 약화
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    
    final_contamination_mask = processed_mask

    # --- 격자 제외 로직 (ROI 내에서 작동) ---
    blurred_gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blurred_gray, 50, 150) # Canny 임계값 (50,150)으로 조정 (이 이미지에 더 적합할 수 있음)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    grid_mask = np.zeros_like(gray, dtype=np.uint8)
    roi_area = roi.shape[:2][0] * roi.shape[:2][1] 
    
    for c in contours:
        area = cv2.contourArea(c)
        if area < 500 or area > 2000: # 면적 범위 (이 이미지 격자 칸 크기 고려)
            continue
        
        epsilon = 0.04 * cv2.arcLength(c, True) 
        approx = cv2.approxPolyDP(c, epsilon, True)

        if len(approx) >= 4 and len(approx) <= 6: 
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            
            if aspect_ratio >= 0.7 and aspect_ratio <= 1.3: 
                cv2.rectangle(grid_mask, (x,y), (x+w, y+h), 255, cv2.FILLED)

    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_CLOSE, np.ones((12,12), np.uint8), iterations=2) 
    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_OPEN, np.ones((9,9), np.uint8), iterations=1)

    final_contamination_mask = cv2.bitwise_and(final_contamination_mask, cv2.bitwise_not(grid_mask))

    # 4) Compute combined ratio
    total_pixels_analyzed = roi.shape[:2][0] * roi.shape[:2][1]
    contaminated_pixels_analyzed = cv2.countNonZero(final_contamination_mask)
    
    if total_pixels_analyzed > 0:
        combined_ratio = (contaminated_pixels_analyzed / total_pixels_analyzed) * 100
    else:
        combined_ratio = 0.0

    # --- 결과 시각화 준비 (파일 저장을 위해) ---
    final_contamination_mask_full = np.zeros_like(full_image[:,:,0], dtype=np.uint8)
    final_contamination_mask_full[y_start : y_start + roi_height, x_start : x_start + roi_width] = final_contamination_mask

    overlay = full_image.copy()
    color_mask = np.zeros_like(full_image, dtype=np.uint8)
    color_mask[final_contamination_mask_full == 255] = [0, 0, 255]

    alpha = 0.5 
    result_image_full = cv2.addWeighted(overlay, 1 - alpha, color_mask, alpha, 0)
    
    # 리사이즈하여 저장
    display_width = 800 
    display_height = int(full_image.shape[:2][0] * (display_width / full_image.shape[:2][1]))
    resized_original = cv2.resize(full_image, (display_width, display_height), interpolation=cv2.INTER_AREA)
    resized_result = cv2.resize(result_image_full, (display_width, display_height), interpolation=cv2.INTER_AREA)
    
    combined_display_image = np.hstack((resized_original, resized_result))

    return combined_ratio, combined_display_image 


# --- 폴더 내 모든 이미지 분석 및 결과 저장 부분 ---
def analyze_and_save_results(folder_path, output_folder, use_roi=True):
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')
    
    # 사용자의 바탕화면 경로를 얻고, 그 안에 'analysis_results' 폴더를 만듭니다.
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    full_output_folder_path = os.path.join(desktop_path, output_folder)

    if not os.path.exists(full_output_folder_path):
        os.makedirs(full_output_folder_path)
        print(f"Created output folder on Desktop: {full_output_folder_path}")
    else:
        print(f"Output folder already exists on Desktop: {full_output_folder_path}")
    
    print(f"--- Analyzing images in folder: {folder_path} (ROI mode: {use_roi}) ---")
    
    results = {}
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            image_full_path = os.path.join(folder_path, filename)
            
            print(f"Analyzing: {filename}")
            
            contamination_percentage, result_display_image = analyze_drain_robust_background_removal(image_full_path, use_roi)
            
            if contamination_percentage is not None and result_display_image is not None:
                results[filename] = contamination_percentage
                
                output_filename = f"result_{os.path.splitext(filename)[0]}_{contamination_percentage:.1f}%.png"
                output_image_path = os.path.join(full_output_folder_path, output_filename) # 바탕화면 경로 사용
                cv2.imwrite(output_image_path, result_display_image)
                print(f"  Result saved to: {output_image_path}")
            else:
                results[filename] = "Error"
                print(f"  Error analyzing {filename}.")
                
    print("\n--- Analysis Complete ---")
    for filename, percentage in results.items():
        if percentage != "Error":
            print(f"{filename}: {percentage:.1f}%")
        else:
            print(f"{filename}: Error")
        
    cv2.destroyAllWindows() 

# --- 함수 호출 예시 ---
# 분석할 이미지 폴더 경로를 지정하세요. (예: 현재 스크립트와 같은 위치에 'images' 폴더가 있다면)
folder_to_analyze = "C:/Users/USER/Desktop/heavy_62" 

# 분석 결과를 저장할 폴더 이름 (바탕화면 안에 이 이름으로 폴더가 생성됩니다)
output_folder_name = "drain_analysis_results_heavy_cropped"

# ROI를 사용하여 분석 (대부분의 경우 추천)
analyze_and_save_results(folder_to_analyze, output_folder_name, use_roi=True)

# ROI를 사용하지 않고 전체 이미지를 분석 (쓰레기가 폴더 전체에 넓게 퍼져있을 때)
# analyze_and_save_results(folder_to_analyze, output_folder_name, use_roi=False)

Created output folder on Desktop: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped
--- Analyzing images in folder: C:/Users/USER/Desktop/heavy_62 (ROI mode: True) ---
Analyzing: heavy_1.jpeg
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_1_57.3%.png
Analyzing: heavy_10.JPG
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_10_52.2%.png
Analyzing: heavy_11.JPG
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_11_71.7%.png
Analyzing: heavy_12.JPG
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_12_73.1%.png
Analyzing: heavy_13.JPG
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_13_49.4%.png
Analyzing: heavy_14.JPG
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_heavy_cropped\result_heavy_14_60.6%.png
Analyzing: heavy_15.JPG
  Result saved to: C:\Users\USER

KeyboardInterrupt: 

In [10]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import os

# analyze_drain_full_image_analysis 함수: ROI 로직 완전 제거, 항상 전체 이미지 분석
def analyze_drain_full_image_analysis(image_path):
    # 1) HSV/BGR thresholds (완전 까만 빈 부분 제외에 초점)
    brown_lower = np.array([0,  20,  50])   # V 10에서 50으로 올림
    brown_upper = np.array([70, 255, 255])
    
    gray_lower  = np.array([0,   40,  80])   # S 10에서 40으로, V 30에서 80으로 대폭 상향
    gray_upper  = np.array([180,100,200])    # V 상한 유지

    pack_lower  = np.array([0,   80,  50])
    pack_upper  = np.array([180,255,255])
    
    green_lower = np.array([30, 20, 10])
    green_upper = np.array([90, 255, 255])

    white_lower_bgr = np.array([100,100,100]) # 60에서 100으로 다시 올림
    white_upper_bgr = np.array([255,255,255])

    hsv_white_lower = np.array([0, 0, 100])     # V 80에서 100으로 올림
    hsv_white_upper = np.array([180, 50, 255])


    # 2) Load image - 이제 이 이미지가 곧 분석 대상이 됩니다.
    img = cv2.imread(image_path) # 변수명을 roi 대신 img로 변경하여 혼동 방지
    if img is None:
        print(f"Cannot load image: {image_path}")
        return None, None 

    h, w = img.shape[:2] # 이미지의 높이와 너비를 직접 사용
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3) Build masks
    mask_brown = cv2.inRange(hsv, brown_lower, brown_upper)
    mask_gray  = cv2.inRange(hsv, gray_lower,  gray_upper)
    soil_mask  = cv2.bitwise_or(mask_brown, mask_gray)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)

    pack_mask  = cv2.inRange(hsv, pack_lower, pack_upper)
    pack_mask  = cv2.bitwise_and(pack_mask, cv2.bitwise_not(soil_mask)) 
    pack_mask = cv2.morphologyEx(pack_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    white_mask_bgr = cv2.inRange(img, white_lower_bgr, white_upper_bgr) 
    white_mask_hsv = cv2.inRange(hsv, hsv_white_lower, hsv_white_upper)
    
    white_candidates_mask = cv2.bitwise_or(white_mask_bgr, white_mask_hsv)

    white_candidates_clean = cv2.morphologyEx(white_candidates_mask,
                                              cv2.MORPH_OPEN,
                                              np.ones((2,2), np.uint8)) 
    
    contours_white, _ = cv2.findContours(white_candidates_clean,
                                         cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)
    
    cig_mask = np.zeros_like(white_candidates_clean) 

    for c in contours_white:
        x,y,w_box,h_box = cv2.boundingRect(c) 
        area = cv2.contourArea(c)

        if (h_box > w_box * 2.5 and area > 80) or (w_box > h_box * 2.5 and area > 80): # 담배 형태 필터링
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)
        elif area > 400 and h_box/w_box < 2 and w_box/h_box < 2: # 휴지 등
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)


    lbp_image = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    lbp_threshold = 60 # LBP 임계값 (빈 공간/그림자 질감 확실히 제외)
    lbp_mask = (lbp_image > lbp_threshold).astype(np.uint8) * 255
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)


    combined_initial_mask = cv2.bitwise_or(soil_mask, pack_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, green_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, cig_mask) 
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, lbp_mask) 

    kernel_small = np.ones((3,3), np.uint8)
    kernel_medium = np.ones((5,5), np.uint8)
    
    processed_mask = cv2.morphologyEx(combined_initial_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_CLOSE, kernel_medium, iterations=1) # 최종 Closing 강도 약화
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    
    final_contamination_mask = processed_mask

    # --- 격자 제외 로직 (이제 입력된 이미지가 격자 이미지라고 가정) ---
    blurred_gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blurred_gray, 50, 150) # Canny 임계값 (50,150)으로 조정 (이 이미지에 더 적합할 수 있음)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    grid_mask = np.zeros_like(gray, dtype=np.uint8)
    total_image_area = h * w # 분석 대상 이미지의 전체 면적
    
    for c in contours:
        area = cv2.contourArea(c)
        if area < 500 or area > 2000: # 면적 범위 (격자 칸 크기 고려, 튜닝 필요)
            continue
        
        epsilon = 0.04 * cv2.arcLength(c, True) 
        approx = cv2.approxPolyDP(c, epsilon, True)

        if len(approx) >= 4 and len(approx) <= 6: 
            x, y, w_box, h_box = cv2.boundingRect(approx) # w, h 변수명 충돌 방지
            aspect_ratio = float(w_box)/h_box
            
            if aspect_ratio >= 0.7 and aspect_ratio <= 1.3: 
                cv2.rectangle(grid_mask, (x,y), (x+w_box, y+h_box), 255, cv2.FILLED)

    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_CLOSE, np.ones((12,12), np.uint8), iterations=2) 
    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_OPEN, np.ones((9,9), np.uint8), iterations=1)

    final_contamination_mask = cv2.bitwise_and(final_contamination_mask, cv2.bitwise_not(grid_mask))

    # 4) Compute combined ratio
    total_pixels_analyzed = h * w 
    contaminated_pixels_analyzed = cv2.countNonZero(final_contamination_mask)
    
    if total_pixels_analyzed > 0:
        combined_ratio = (contaminated_pixels_analyzed / total_pixels_analyzed) * 100
    else:
        combined_ratio = 0.0

    print(f"Combined contamination ratio: {combined_ratio:.1f}%")

    # --- 결과 시각화 준비 (파일 저장을 위해) ---
    overlay = img.copy() # img가 원본 이미지 (roi 역할)
    color_mask = np.zeros_like(img, dtype=np.uint8)
    color_mask[final_contamination_mask == 255] = [0, 0, 255]

    alpha = 0.5 
    result_image_full = cv2.addWeighted(overlay, 1 - alpha, color_mask, alpha, 0)
    
    display_width = 800 
    display_height = int(h * (display_width / w))
    resized_original = cv2.resize(img, (display_width, display_height), interpolation=cv2.INTER_AREA)
    resized_result = cv2.resize(result_image_full, (display_width, display_height), interpolation=cv2.INTER_AREA)
    
    combined_display_image = np.hstack((resized_original, resized_result))

    return combined_ratio, combined_display_image 


# --- 폴더 내 모든 이미지 분석 및 결과 저장 부분 ---
# 이 함수에서는 use_roi 매개변수를 완전히 제거합니다.
def analyze_and_save_results_no_roi(folder_path, output_folder): 
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')
    
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    full_output_folder_path = os.path.join(desktop_path, output_folder)

    if not os.path.exists(full_output_folder_path):
        os.makedirs(full_output_folder_path)
        print(f"Created output folder on Desktop: {full_output_folder_path}")
    else:
        print(f"Output folder already exists on Desktop: {full_output_folder_path}")
    
    print(f"--- Analyzing images in folder: {folder_path} (Full Image Analysis) ---") # 메시지 변경
    
    results = {}
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            image_full_path = os.path.join(folder_path, filename)
            
            print(f"Analyzing: {filename}")
            
            # analyze_drain_full_image_analysis 함수 호출 (use_roi 매개변수 없음)
            contamination_percentage, result_display_image = analyze_drain_full_image_analysis(image_full_path)
            
            if contamination_percentage is not None and result_display_image is not None:
                results[filename] = contamination_percentage
                
                output_filename = f"result_{os.path.splitext(filename)[0]}_{contamination_percentage:.1f}%.png"
                output_image_path = os.path.join(full_output_folder_path, output_filename)
                cv2.imwrite(output_image_path, result_display_image)
                print(f"  Result saved to: {output_image_path}")
            else:
                results[filename] = "Error"
                print(f"  Error analyzing {filename}.")
                
    print("\n--- Analysis Complete ---")
    for filename, percentage in results.items():
        if percentage != "Error":
            print(f"{filename}: {percentage:.1f}%")
        else:
            print(f"{filename}: Error")
        
    cv2.destroyAllWindows() 

# --- 함수 호출 예시 ---
# 분석할 이미지 폴더 경로를 지정하세요. (예: 현재 스크립트와 같은 위치에 'images' 폴더가 있다고 가정)
folder_to_analyze = "C:/Users/USER/Desktop/heavy_62" 

# 분석 결과를 저장할 폴더 이름 (바탕화면 안에 이 이름으로 폴더가 생성됩니다)
output_folder_name = "drain_analysis_results_tuned"

# 수동 크롭된 이미지를 가정하고 전체 이미지 분석
analyze_and_save_results_no_roi(folder_to_analyze, output_folder_name)

Created output folder on Desktop: C:\Users\USER\Desktop\drain_analysis_results_tuned
--- Analyzing images in folder: C:/Users/USER/Desktop/heavy_62 (Full Image Analysis) ---
Analyzing: heavy_1.jpeg
Combined contamination ratio: 86.6%
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_tuned\result_heavy_1_86.6%.png
Analyzing: heavy_10.JPG
Combined contamination ratio: 55.7%
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_tuned\result_heavy_10_55.7%.png
Analyzing: heavy_11.JPG
Combined contamination ratio: 72.5%
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_tuned\result_heavy_11_72.5%.png
Analyzing: heavy_12.JPG
Combined contamination ratio: 77.8%
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_tuned\result_heavy_12_77.8%.png
Analyzing: heavy_13.JPG
Combined contamination ratio: 44.4%
  Result saved to: C:\Users\USER\Desktop\drain_analysis_results_tuned\result_heavy_13_44.4%.png
Analyzing: heavy_14.JPG
Combined contamination 